In [1]:
import torch
import torch.nn as nn
import torch.nn.init as init
import random
import copy
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import DeepTreeEnsemble
import torch.nn.functional as F
from importlib import reload
import pandas as pd
reload(DeepTreeEnsemble)
from DeepTreeEnsemble import DeepTreeEnsemble, create_dataloaders_for_dataset, create_imageloaders_for_dataset
# if this ^ doesn't work, try 'pip install -e .' in the root directory of it. Make sure you are in 
# the desired environment. 

In [2]:
train_dataloader, test_dataloader, input_size, output_size = create_imageloaders_for_dataset('MNIST')

Files already downloaded and verified


In [4]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_layers, output_size, dropout=.5):
        super(MLP, self).__init__()
        self.layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList()

        # Create the first hidden layer
        self.layers.append(nn.Linear(input_size, hidden_layers[0]))
        self.batch_norms.append(nn.BatchNorm1d(hidden_layers[0]))

        # Create additional hidden layers
        for i in range(1, len(hidden_layers)):
            self.layers.append(nn.Linear(hidden_layers[i-1], hidden_layers[i]))
            self.batch_norms.append(nn.BatchNorm1d(hidden_layers[i]))

        # Output layer
        self.output = nn.Linear(hidden_layers[-1], output_size)

        # Dropout layer
        self.dropout = nn.Dropout(dropout)


    def forward(self, x):
        i = 0
        for layer, batch_norm in zip(self.layers, self.batch_norms):
            # x = F.relu(batch_norm(layer(x)))
            i=i+1
            x = self.dropout(x)
            # print('Layer dtype: ', layer.weight.dtype)
            x = x.to(torch.float32) # <-- area of concern
            # print('tensor dytpe: ', x.dtype)
            # layer.weight = layer.weight.to(torch.float32)
            x = F.relu(batch_norm(layer(x)))
            
            # print(f'dtype is {x.dtype} after layer {i}')

        x = self.output(x)
        return x

In [5]:
hidden_layers = [25, 25]
multiLayerPerceptron = MLP(input_size=input_size, hidden_layers=hidden_layers, output_size=output_size+1, dropout=.5) #hidden size default at 100
print("got here")
# Fill DTE params
modelSaveDirectory = './saved_models/'
DTEmlp4b = DeepTreeEnsemble(task_name='classification',
                                model_arch=multiLayerPerceptron,
                                model_dir=modelSaveDirectory, 
                                base_number=4,
                                epochs=10,
                                train_dataloader=train_dataloader, 
                                test_dataloader=test_dataloader,
                                learning_rate=0.0001)

got here


In [6]:
def run_experiment(DTE, trails, output_directory):
    results = pd.DataFrame()

    for i in range(trails):
        df1 = DTE.train_DTE()
        df2 = DTE.single_model()
        combined_df = pd.concat([df1, df2], axis=1)
        results = pd.concat([results, combined_df], axis=0)

    results.to_csv(output_directory, index=False)

In [7]:
output_directory = './output.csv'
run_experiment(DTEmlp4b, trails=5, output_directory=output_directory)

Randomly initializing 4 models
Training Model with id 1
in multi


Training Process: 100%|██████████| 10/10 [01:45<00:00, 10.50s/it]


Total Epochs: 10
Train loss start: 2.4064688682556152, Train loss end: 1.7962603569030762
Highest Mean Train Accuracy (over epoch): 0.573724452014218
Test loss start: 2.1672325134277344, Test loss end: 1.731744647026062
Highest mean Test Accuracy (over epoch): 0.7231715425531915
 Test F1 score at highest accuracy: 0.530062355407572
./saved_models/model_1.pth 

Training Model with id 2
in multi


Training Process: 100%|██████████| 10/10 [01:41<00:00, 10.17s/it]


Total Epochs: 10
Train loss start: 2.401355743408203, Train loss end: 1.9344689846038818
Highest Mean Train Accuracy (over epoch): 0.5830976007109004
Test loss start: 2.122525215148926, Test loss end: 1.627313256263733
Highest mean Test Accuracy (over epoch): 0.776529255319149
 Test F1 score at highest accuracy: 0.5639517594193757
./saved_models/model_2.pth 

Training Model with id 3
in multi


...

Total Epochs: 10
Train loss start: 1.8341199159622192, Train loss end: 1.9060267210006714
Highest Mean Train Accuracy (over epoch): 0.7354765254739336
Test loss start: 1.7119882106781006, Test loss end: 1.5504266023635864
Highest mean Test Accuracy (over epoch): 0.8930684840425532
 Test F1 score at highest accuracy: 0.7373557868357229
./saved_models/model_8.pth 

Training Model with id 6
in multi


Training Process: 100%|██████████| 10/10 [01:39<00:00,  9.99s/it]


Total Epochs: 10
Train loss start: 1.767219066619873, Train loss end: 1.7720463275909424
Highest Mean Train Accuracy (over epoch): 0.74700273992891
Test loss start: 1.6875070333480835, Test loss end: 1.551195740699768
Highest mean Test Accuracy (over epoch): 0.8966921542553191
 Test F1 score at highest accuracy: 0.7487135684586669
./saved_models/model_6.pth 

 Maximum accuracy: 0.8966921542553191, F1: 0.7487135684586669, Test: 1.6508420420453904, Train: 1.8018218632911054
in multi


Training Process:  79%|███████▊  | 55/70 [11:50<03:13, 12.92s/it]


KeyboardInterrupt: 

In [8]:
hidden_layers = [25, 25]
multiLayerPerceptron = MLP(input_size=input_size, hidden_layers=hidden_layers, output_size=output_size+1, dropout=.5) #hidden size default at 100

# Fill DTE params
modelSaveDirectory = './saved_models/'
DTEmlp8b = DeepTreeEnsemble(task_name='classification',
                                model_arch=multiLayerPerceptron,
                                model_dir=modelSaveDirectory, 
                                base_number=4,
                                epochs=10,
                                train_dataloader=train_dataloader, 
                                test_dataloader=test_dataloader,
                                learning_rate=0.0001)

In [4]:
output_directory = './outputBase8.csv'
run_experiment(DTEmlp8b, trails=5, output_directory=output_directory)

NameError: name 'DTEmlp8b' is not defined